# IA prediction d'humeur BERT

instalation des transdormers

In [2]:
%pip install torch                
%pip install transformers 
%pip install pandas 
%pip install scikit-learn
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from dotenv import load_dotenv
from pathlib import Path
import os




# 1. Charger les données


In [4]:
import pandas as pd
import json

load_dotenv()

FICHIER_REVIEWS = os.getenv("INPUT_REVIEWS")


# Méthode 1 : Avec pandas (recommandé)
try:
    data = pd.read_json(FICHIER_REVIEWS, lines=True)
    print(f"✓ Données chargées : {len(data)} lignes")
    print(f"✓ Colonnes : {list(data.columns)}")
    print(data.head())
except FileNotFoundError:
    print(f"✗ Fichier introuvable : {FICHIER_REVIEWS}")
    print("Vérifiez que le fichier existe dans le dossier 'data' du répertoire parent")
except Exception as e:
    print(f"✗ Erreur : {e}")


✓ Données chargées : 1000000 lignes
✓ Colonnes : ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
                review_id                 user_id             business_id  \
0  J5Q1gH4ACCj6CtQG7Yom7g  56gL9KEJNHiSDUoyjk2o3Q  8yR12PNSMo6FBYx1u5KPlw   
1  HlXP79ecTquSVXmjM10QxQ  bAt9OUFX9ZRgGLCXG22UmA  pBNucviUkNsiqhJv5IFpjg   
2  JBBULrjyGx6vHto2osk_CQ  NRHPcLq2vGWqgqwVugSgnQ  8sf9kv6O4GgEb0j1o22N1g   
3  U9-43s8YUl6GWBFCpxUGEw  PAxc0qpqt5c2kA0rjDFFAg  XwepyB7KjJ-XGJf0vKc6Vg   
4  8T8EGa_4Cj12M6w8vRgUsQ  BqPR1Dp5Rb_QYs9_fz9RiA  prm5wvpp0OHJBlrvTj9uOg   

   stars  useful  funny  cool  \
0      2       1      0     0   
1      5       0      0     0   
2      5       0      0     0   
3      4       0      0     0   
4      5       0      0     0   

                                                text                date  
0  Went for lunch and found that my burger was me... 2018-04-04 21:09:53  
1  I needed a new tires for my wife's car. They

In [5]:
import re
import pandas as pd

def load_yelp_dataset(path):
    """
    Charge le dataset Yelp depuis un fichier JSONL
    """
    # Charger directement avec pandas (beaucoup plus simple et rapide)
    data = pd.read_json(path, lines=True)
    
    # Ne garder que les colonnes nécessaires
    df = pd.DataFrame({
        "texte": data["text"],
        "stars": data["stars"]
    })
    
    print(f"✓ Données chargées : {len(df)} lignes")
    print(f"✓ Distribution des étoiles :")
    print(df["stars"].value_counts().sort_index())
    
    return df


In [6]:
df = load_yelp_dataset(FICHIER_REVIEWS)
print(len(df))
print(df.head())


✓ Données chargées : 1000000 lignes
✓ Distribution des étoiles :
stars
1    153057
2     77630
3     98714
4    207953
5    462646
Name: count, dtype: int64
1000000
                                               texte  stars
0  Went for lunch and found that my burger was me...      2
1  I needed a new tires for my wife's car. They h...      5
2  Jim Woltman who works at Goleta Honda is 5 sta...      5
3  Been here a few times to get some shrimp.  The...      4
4  This is one fantastic place to eat whether you...      5


In [7]:
df = df.sample(200000, random_state=42)


In [8]:
def stars_to_label(stars):
    if stars <= 2:
        return 0   # negatif
    elif stars == 3:
        return 1   # neutre
    else:
        return 2   # positif

df["label"] = df["stars"].apply(stars_to_label)


# 2. Train / Test split


In [9]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["texte"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# 3. Création voc


In [10]:
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import torch.nn as nn


def build_vocab(texts, max_vocab_size=10000):
    counter = Counter()
    for text in texts:
        counter.update(text.lower().split())
        
    most_common = counter.most_common(max_vocab_size - 2)
    
    vocab = {"<PAD>": 0, "<UNK>": 1}
    for i, (word, _) in enumerate(most_common):
        vocab[word] = i + 2
        
    return vocab

vocab = build_vocab(train_texts)

def encode_text(text, vocab):
    return torch.tensor([
        vocab.get(word, vocab["<UNK>"])
        for word in text.lower().split()
    ])




# 4. Dataset PyTorch


In [11]:
class MoodDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoded = encode_text(self.texts[idx], self.vocab)
        return encoded, torch.tensor(self.labels[idx])

def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    return texts_padded, torch.tensor(labels)

train_dataset = MoodDataset(train_texts, train_labels, vocab)
test_dataset = MoodDataset(test_texts, test_labels, vocab)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)


# 5. Modèle


In [12]:

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextCNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        self.conv = nn.Conv1d(
            in_channels=embed_dim,
            out_channels=100,
            kernel_size=3
        )
        
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x)           # (batch, seq_len, embed_dim)
        x = x.permute(0, 2, 1)          # (batch, embed_dim, seq_len)
        x = torch.relu(self.conv(x))
        x = self.pool(x).squeeze(2)
        x = self.dropout(x)
        x = self.fc(x)
        return x



# 6. Initialisation


In [13]:
model = TextCNN(
    vocab_size=len(vocab),
    embed_dim=100,
    num_classes=3
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()



# 7. Entraînement


In [14]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.5323
Epoch 2, Loss: 0.3952
Epoch 3, Loss: 0.3619
Epoch 4, Loss: 0.3423
Epoch 5, Loss: 0.3283


# 8. Évaluation


In [15]:
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for texts, labels in test_loader:
        texts = texts.to(device)
        outputs = model(texts)
        preds = torch.argmax(outputs, dim=1)
        
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.numpy())

print(classification_report(true_labels, predictions))


              precision    recall  f1-score   support

           0       0.85      0.85      0.85      9262
           1       0.55      0.28      0.37      3895
           2       0.91      0.97      0.94     26843

    accuracy                           0.87     40000
   macro avg       0.77      0.70      0.72     40000
weighted avg       0.86      0.87      0.86     40000



# 9. test manuel

In [17]:
def predict_sentence(sentence, model, vocab, device):
    
    model.eval()
    
    encoded = encode_text(sentence, vocab)
    
    encoded = encoded.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(encoded)
        prediction = torch.argmax(output, dim=1).item()
    
    labels = {
        0: "NEGATIF",
        1: "NEUTRE",
        2: "POSITIF"
    }
    
    print("Phrase:", sentence)
    print("Prediction:", labels[prediction])








In [18]:
predict_sentence("I love this restaurant", model, vocab, device)

predict_sentence("This is terrible", model, vocab, device)

predict_sentence("It is ok", model, vocab, device)

Phrase: I love this restaurant
Prediction: POSITIF
Phrase: This is terrible
Prediction: NEGATIF
Phrase: It is ok
Prediction: NEGATIF
